In [6]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)   
pd.set_option('display.max_rows', None)      
import re

In [7]:
df = pd.read_excel("Dataset/combined_database.xlsx")

In [8]:
df.head()

,site_no,soil_type,texture,year,duration,material_name,type,material_id,material_code,weight_loss_oz,specimen_status,city,state,internal_drainage,resistivity,pH,pH_type,total_acidity,Na+K,Ca,Mg,CO3,HCO3,Cl,SO4,mean_temp,annual_precipitation,moisture,air_pore_space,specific_gravity,volume_shrinkage,table,iron_type,areation,region
0,1,Allis silt loam,Clay,1922,1.0,Wrought iron,1.5-inch pipe,1,b,1.2,intact,Cleveland,OH,P,1215.0,7.0,Neutral,14.4,0.72,0.25,0.43,0.0,0.09,0.09,0.83,49.2,33.8,28.6,1.1,NaN,6.6,13,Wrought,Poor,Northern
1,1,Allis silt loam,Clay,1922,3.6,Wrought iron,1.5-inch pipe,1,b,4.0,intact,Cleveland,OH,P,1215.0,7.0,Neutral,14.4,0.72,0.25,0.43,0.0,0.09,0.09,0.83,49.2,33.8,28.6,1.1,NaN,6.6,13,Wrought,Poor,Northern
2,1,Allis silt loam,Clay,1922,5.5,Wrought iron,1.5-inch pipe,1,b,5.5,intact,Cleveland,OH,P,1215.0,7.0,Neutral,14.4,0.72,0.25,0.43,0.0,0.09,0.09,0.83,49.2,33.8,28.6,1.1,NaN,6.6,13,Wrought,Poor,Northern
3,1,Allis silt loam,Clay,1922,7.7,Wrought iron,1.5-inch pipe,1,b,6.8,intact,Cleveland,OH,P,1215.0,7.0,Neutral,14.4,0.72,0.25,0.43,0.0,0.09,0.09,0.83,49.2,33.8,28.6,1.1,NaN,6.6,13,Wrought,Poor,Northern
4,1,Allis silt loam,Clay,1922,9.6,Wrought iron,1.5-inch pipe,1,b,9.4,intact,Cleveland,OH,P,1215.0,7.0,Neutral,14.4,0.72,0.25,0.43,0.0,0.09,0.09,0.83,49.2,33.8,28.6,1.1,NaN,6.6,13,Wrought,Poor,Northern


In [8]:
df['material_id'].unique()

array([ 1,  3, 12, 14, 16, 17, 18, 26, 13,  4, 19,  7, 15,  8, 24, 25, 35,
       41, 37, 45, 46, 32, 33, 47, 34, 43, 29, 31, 42, 30, 38, 44, 36, 64,
       60, 67, 65, 62, 63, 70, 75, 74, 76, 79, 80, 68, 72, 73, 78, 77])

#### **Calculate Number of Days**


In [9]:
def calculate_days_from_year(start_year, duration_years):
    start_date = datetime(start_year, 1, 1)
    approx_days = duration_years * 365.25
    end_date = start_date + timedelta(days=approx_days)
    exact_days = (end_date - start_date).days
    return round(approx_days, 2), exact_days

results = df.apply(
    lambda row: calculate_days_from_year(int(row['year']), float(row['duration']))
    if pd.notnull(row['year']) and pd.notnull(row['duration']) else (None, None, None),
    axis=1, result_type='expand'
)
    
df[['approximate_days', 'exact_days']] = results

#### **Calculate weight loss in g/m2**


In [10]:
df['weight_loss_oz'] = pd.to_numeric(df['weight_loss_oz'], errors='coerce')
df['weight_loss_g'] = df['weight_loss_oz'] * 305.1517
df['weight_loss_g'] = df['weight_loss_g'].round(2)

#### **Calculate Corrosion Rate**


In [11]:
df['corrosion_rate'] = df['weight_loss_g'] / df['approximate_days']
df['corrosion_rate'] = df['corrosion_rate'].round(4)

In [12]:
df.head()

,site_no,soil_type,texture,year,duration,material_name,type,material_id,material_code,weight_loss_oz,specimen_status,city,state,internal_drainage,resistivity,pH,pH_type,total_acidity,Na+K,Ca,Mg,CO3,HCO3,Cl,SO4,mean_temp,annual_precipitation,moisture,air_pore_space,specific_gravity,volume_shrinkage,table,iron_type,areation,region,approximate_days,exact_days,weight_loss_g,corrosion_rate
0,1,Allis silt loam,Clay,1922,1.0,Wrought iron,1.5-inch pipe,1,b,1.2,intact,Cleveland,OH,P,1215.0,7.0,Neutral,14.4,0.72,0.25,0.43,0.0,0.09,0.09,0.83,49.2,33.8,28.6,1.1,NaN,6.6,13,Wrought,Poor,Northern,365.25,365.0,366.18,1.0025
1,1,Allis silt loam,Clay,1922,3.6,Wrought iron,1.5-inch pipe,1,b,4.0,intact,Cleveland,OH,P,1215.0,7.0,Neutral,14.4,0.72,0.25,0.43,0.0,0.09,0.09,0.83,49.2,33.8,28.6,1.1,NaN,6.6,13,Wrought,Poor,Northern,1314.90,1314.0,1220.61,0.9283
2,1,Allis silt loam,Clay,1922,5.5,Wrought iron,1.5-inch pipe,1,b,5.5,intact,Cleveland,OH,P,1215.0,7.0,Neutral,14.4,0.72,0.25,0.43,0.0,0.09,0.09,0.83,49.2,33.8,28.6,1.1,NaN,6.6,13,Wrought,Poor,Northern,2008.88,2008.0,1678.33,0.8355
3,1,Allis silt loam,Clay,1922,7.7,Wrought iron,1.5-inch pipe,1,b,6.8,intact,Cleveland,OH,P,1215.0,7.0,Neutral,14.4,0.72,0.25,0.43,0.0,0.09,0.09,0.83,49.2,33.8,28.6,1.1,NaN,6.6,13,Wrought,Poor,Northern,2812.43,2812.0,2075.03,0.7378
4,1,Allis silt loam,Clay,1922,9.6,Wrought iron,1.5-inch pipe,1,b,9.4,intact,Cleveland,OH,P,1215.0,7.0,Neutral,14.4,0.72,0.25,0.43,0.0,0.09,0.09,0.83,49.2,33.8,28.6,1.1,NaN,6.6,13,Wrought,Poor,Northern,3506.40,3506.0,2868.43,0.8181


In [8]:
df.to_csv('database.csv', index=False)

#### **Combine Database with specimen details**


In [20]:
df2 = pd.read_csv('composition_clr.csv')

In [21]:
df1 = pd.read_csv('database.csv')


In [16]:
df1.head()

,site_no,soil_type,texture,year,duration,material_name,type,material_id,material_code,weight_loss_oz,specimen_status,city,state,internal_drainage,resistivity,pH,pH_type,total_acidity,Na+K,Ca,Mg,CO3,HCO3,Cl,SO4,mean_temp,annual_precipitation,moisture,air_pore_space,specific_gravity,volume_shrinkage,table,iron_type,areation,region,approximate_days,exact_days,weight_loss_g,corrosion_rate
0,1,Allis silt loam,Clay,1922,1.0,Wrought iron,1.5-inch pipe,1,b,1.2,intact,Cleveland,OH,P,1215.0,7.0,Neutral,14.4,0.72,0.25,0.43,0.0,0.09,0.09,0.83,49.2,33.8,28.6,1.1,NaN,6.6,13,Wrought,Poor,Northern,365.25,365.0,366.18,1.0025
1,1,Allis silt loam,Clay,1922,3.6,Wrought iron,1.5-inch pipe,1,b,4.0,intact,Cleveland,OH,P,1215.0,7.0,Neutral,14.4,0.72,0.25,0.43,0.0,0.09,0.09,0.83,49.2,33.8,28.6,1.1,NaN,6.6,13,Wrought,Poor,Northern,1314.90,1314.0,1220.61,0.9283
2,1,Allis silt loam,Clay,1922,5.5,Wrought iron,1.5-inch pipe,1,b,5.5,intact,Cleveland,OH,P,1215.0,7.0,Neutral,14.4,0.72,0.25,0.43,0.0,0.09,0.09,0.83,49.2,33.8,28.6,1.1,NaN,6.6,13,Wrought,Poor,Northern,2008.88,2008.0,1678.33,0.8355
3,1,Allis silt loam,Clay,1922,7.7,Wrought iron,1.5-inch pipe,1,b,6.8,intact,Cleveland,OH,P,1215.0,7.0,Neutral,14.4,0.72,0.25,0.43,0.0,0.09,0.09,0.83,49.2,33.8,28.6,1.1,NaN,6.6,13,Wrought,Poor,Northern,2812.43,2812.0,2075.03,0.7378
4,1,Allis silt loam,Clay,1922,9.6,Wrought iron,1.5-inch pipe,1,b,9.4,intact,Cleveland,OH,P,1215.0,7.0,Neutral,14.4,0.72,0.25,0.43,0.0,0.09,0.09,0.83,49.2,33.8,28.6,1.1,NaN,6.6,13,Wrought,Poor,Northern,3506.40,3506.0,2868.43,0.8181


In [22]:
df2.head()

,material_id,C (%),Si (%),Mn (%),S (%),P (%),Cr (%),Ni (%),Cu (%),Mo (%),Fe (%),C_clr,Si_clr,Mn_clr,S_clr,P_clr,Cr_clr,Ni_clr,Cu_clr,Mo_clr
0,1,0.030,0.150,0.005,0.023,0.145,0.0,0.0,0.02,0.0,99.627,5.219872,6.829310,3.428114,4.954169,6.795409,-10.680427,-10.680427,4.814407,-10.680427
1,3,0.020,0.150,0.033,0.022,0.195,0.0,0.0,0.03,0.0,99.550,4.514197,6.529100,5.014973,4.609508,6.791464,-10.792968,-10.792968,4.919662,-10.792968
2,4,0.020,0.150,0.033,0.022,0.195,0.0,0.0,0.03,0.0,99.550,4.514197,6.529100,5.014973,4.609508,6.791464,-10.792968,-10.792968,4.919662,-10.792968
3,7,0.016,0.100,0.029,0.018,0.160,0.0,0.0,0.00,0.0,99.677,6.312757,8.145338,6.907464,6.430540,8.615342,-9.102860,-9.102860,-9.102860,-9.102860
4,8,0.017,0.125,0.041,0.018,0.106,0.0,0.0,0.00,0.0,99.693,6.371705,8.366805,7.252063,6.428863,8.201930,-9.155342,-9.155342,-9.155342,-9.155342


In [23]:
df_final = df1.merge(df2, on='material_id', how='left')

#### **Year Bins**


In [13]:
bins = np.arange(0, 22, 2)  # end at 22 to include 20 in the last bin
labels = [f"{i}-{i+2}" for i in range(0, 20, 2)]

# Create bin column
df_final['duration_bin'] = pd.cut(df_final['duration'], bins=bins, labels=labels, right=False)
df_final['duration_bin'] = df_final['duration_bin'].astype(str)

In [24]:
df_final.to_csv('final_database1.csv', index=False)

In [25]:
df_final.head()

,site_no,soil_type,texture,year,duration,material_name,type,material_id,material_code,weight_loss_oz,specimen_status,city,state,internal_drainage,resistivity,pH,pH_type,total_acidity,Na+K,Ca,Mg,CO3,HCO3,Cl,SO4,mean_temp,annual_precipitation,moisture,air_pore_space,specific_gravity,volume_shrinkage,table,iron_type,areation,region,approximate_days,exact_days,weight_loss_g,corrosion_rate,C (%),Si (%),Mn (%),S (%),P (%),Cr (%),Ni (%),Cu (%),Mo (%),Fe (%),C_clr,Si_clr,Mn_clr,S_clr,P_clr,Cr_clr,Ni_clr,Cu_clr,Mo_clr
0,1,Allis silt loam,Clay,1922,1.0,Wrought iron,1.5-inch pipe,1,b,1.2,intact,Cleveland,OH,P,1215.0,7.0,Neutral,14.4,0.72,0.25,0.43,0.0,0.09,0.09,0.83,49.2,33.8,28.6,1.1,NaN,6.6,13,Wrought,Poor,Northern,365.25,365.0,366.18,1.0025,0.03,0.15,0.005,0.023,0.145,0.0,0.0,0.02,0.0,99.627,5.219872,6.82931,3.428114,4.954169,6.795409,-10.680427,-10.680427,4.814407,-10.680427
1,1,Allis silt loam,Clay,1922,3.6,Wrought iron,1.5-inch pipe,1,b,4.0,intact,Cleveland,OH,P,1215.0,7.0,Neutral,14.4,0.72,0.25,0.43,0.0,0.09,0.09,0.83,49.2,33.8,28.6,1.1,NaN,6.6,13,Wrought,Poor,Northern,1314.90,1314.0,1220.61,0.9283,0.03,0.15,0.005,0.023,0.145,0.0,0.0,0.02,0.0,99.627,5.219872,6.82931,3.428114,4.954169,6.795409,-10.680427,-10.680427,4.814407,-10.680427
2,1,Allis silt loam,Clay,1922,5.5,Wrought iron,1.5-inch pipe,1,b,5.5,intact,Cleveland,OH,P,1215.0,7.0,Neutral,14.4,0.72,0.25,0.43,0.0,0.09,0.09,0.83,49.2,33.8,28.6,1.1,NaN,6.6,13,Wrought,Poor,Northern,2008.88,2008.0,1678.33,0.8355,0.03,0.15,0.005,0.023,0.145,0.0,0.0,0.02,0.0,99.627,5.219872,6.82931,3.428114,4.954169,6.795409,-10.680427,-10.680427,4.814407,-10.680427
3,1,Allis silt loam,Clay,1922,7.7,Wrought iron,1.5-inch pipe,1,b,6.8,intact,Cleveland,OH,P,1215.0,7.0,Neutral,14.4,0.72,0.25,0.43,0.0,0.09,0.09,0.83,49.2,33.8,28.6,1.1,NaN,6.6,13,Wrought,Poor,Northern,2812.43,2812.0,2075.03,0.7378,0.03,0.15,0.005,0.023,0.145,0.0,0.0,0.02,0.0,99.627,5.219872,6.82931,3.428114,4.954169,6.795409,-10.680427,-10.680427,4.814407,-10.680427
4,1,Allis silt loam,Clay,1922,9.6,Wrought iron,1.5-inch pipe,1,b,9.4,intact,Cleveland,OH,P,1215.0,7.0,Neutral,14.4,0.72,0.25,0.43,0.0,0.09,0.09,0.83,49.2,33.8,28.6,1.1,NaN,6.6,13,Wrought,Poor,Northern,3506.40,3506.0,2868.43,0.8181,0.03,0.15,0.005,0.023,0.145,0.0,0.0,0.02,0.0,99.627,5.219872,6.82931,3.428114,4.954169,6.795409,-10.680427,-10.680427,4.814407,-10.680427


In [16]:
df_final.shape

(5654, 61)

### **Soil Texture of Sites**


In [ ]:
import pandas as pd

def classify_soil_texture(sand, silt, clay):
    total = sand + silt + clay
    if total == 0:
        return "Invalid"

   
    sand = sand / total * 100
    silt = silt / total * 100
    clay = clay / total * 100

    if clay >= 40:
        if silt >= 40:
            return "Silty Clay"
        elif sand >= 45:
            return "Sandy Clay"
        else:
            return "Clay"
    elif clay >= 35:
        if sand >= 45:
            return "Sandy Clay"
        elif silt >= 40:
            return "Silty Clay"
        else:
            return "Clay"
    elif clay >= 27:
        if silt >= 40:
            return "Silty Clay Loam"
        elif sand >= 45:
            return "Sandy Clay Loam"
        else:
            return "Clay Loam"
    elif clay >= 20:
        if sand >= 52:
            return "Sandy Clay Loam"
        elif 28 <= silt < 50 and 20 <= sand <= 45:
            return "Clay Loam"
        else:
            return "Silty Clay Loam"
    elif clay >= 7:
        if silt >= 50 and clay < 27:
            return "Silt Loam"
        elif silt >= 80 and clay < 12:
            return "Silt"
        elif sand >= 52 and clay < 20:
            return "Sandy Loam"
        elif 43 <= sand < 52:
            return "Loam"
        else:
            return "Silt Loam"
    else:
        if sand >= 85:
            return "Sand"
        elif sand >= 70:
            return "Loamy Sand"
        else:
            return "Sandy Loam"

df = pd.read_csv("./Dataset/sites.csv")
df["soil_texture"] = df.apply(lambda row: classify_soil_texture(row["sand"], row["silt"], row["clay"]), axis=1)
df.to_csv("classified_soil_textures.csv", index=False)


### **North and South Regions**


In [5]:
import pandas as pd

# Mapping of state abbreviations to full names
state_abbrev_to_name = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas',
    'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware',
    'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho',
    'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas',
    'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland',
    'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi',
    'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada',
    'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NY': 'New York',
    'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma',
    'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
    'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah',
    'VT': 'Vermont', 'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia',
    'WI': 'Wisconsin', 'WY': 'Wyoming', 'DC': 'District of Columbia'
}

# Lists of states based on pipe-laying regions
northern_states = {
    'Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 'Connecticut', 'Rhode Island',
    'New York', 'New Jersey', 'Pennsylvania', 'Ohio', 'Michigan', 'Wisconsin',
    'Minnesota', 'Iowa', 'Illinois', 'Indiana', 'North Dakota', 'South Dakota',
    'Nebraska', 'Montana', 'Wyoming', 'Colorado', 'Idaho', 'Washington', 'Oregon'
}

southern_states = {
    'Florida', 'Georgia', 'Alabama', 'Mississippi', 'Louisiana', 'Texas',
    'Arkansas', 'Oklahoma', 'New Mexico', 'Arizona', 'Nevada', 'South Carolina',
    'North Carolina', 'Tennessee', 'Kentucky', 'Virginia', 'California',
    'Hawaii', 'District of Columbia', 'West Virginia'
}

# Function to classify a state
def classify_region(state_abbrev):
    full_name = state_abbrev_to_name.get(state_abbrev.upper(), "Unknown")
    if full_name in northern_states:
        region = "Northern"
    elif full_name in southern_states:
        region = "Southern"
    else:
        region = "Transitional or Unknown"
    return pd.Series([full_name, region])

# Read the CSV
df = pd.read_csv("./Dataset/sites.csv")  # Your input CSV file

# Apply the classification
df[['state_full', 'region']] = df['State'].apply(classify_region)

# Save to new CSV
df.to_csv("output_classified_states.csv", index=False)

print("Classification complete. Saved to output_classified_states.csv.")


Classification complete. Saved to output_classified_states.csv.


In [3]:
import pandas as pd
import numpy as np
from pathlib import Path

# ---------- CLR helpers ----------
def clr(X, pseudocount=1e-12):
    X = np.asarray(X, dtype=float)
    if (X < 0).any():
        raise ValueError("CLR requires nonnegative values.")
    # close rows to sum=1
    row_sums = X.sum(axis=1, keepdims=True)
    if (row_sums == 0).any():
        raise ValueError("Found rows that sum to 0; cannot apply CLR.")
    X = X / row_sums
    # handle zeros
    X = X + pseudocount
    X = X / X.sum(axis=1, keepdims=True)
    logX = np.log(X)
    gm = logX.mean(axis=1, keepdims=True)
    return logX - gm

def clr_df(df, cols, pseudocount=1e-12, drop_one_for_linear=False, suffix="_clr"):
    X = df[cols].to_numpy(dtype=float)
    Z = clr(X, pseudocount=pseudocount)
    out = df.copy()
    names = [f"{c}{suffix}" for c in cols]
    if drop_one_for_linear:
        Z = Z[:, :-1]
        names = names[:-1]
    out[names] = Z
    return out

# ---------- Configure these ----------                  # change if needed
id_cols  = ["material_no"]             # keep these as identifiers in the output
pseudoc  = 1e-8                        # bigger if you have many zeros (e.g., 1e-6)
                       # True for linear/elastic-net models

# ---------- Load & clean ----------
df = pd.read_csv('composition.csv')

# Convert "Trace" → 0.005, coerce non-numeric in comp columns to NaN→0
# (composition columns are assumed to contain '%' in their header)
comp_cols = [c for c in df.columns if "%" in c]
df[comp_cols] = df[comp_cols].replace("Trace", 0.005)
for c in comp_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0.0)

# Optional: drop Fe (balance adds no info and creates collinearity)
comp_cols_no_fe = [c for c in comp_cols if c.strip().lower() not in {"fe", "fe (%)", "fe%"}]
if len(comp_cols_no_fe) < 2:
    raise ValueError("Need at least 2 non‑Fe composition columns for CLR.")

 

# ---------- Apply CLR ----------
df_out = clr_df(df, cols=comp_cols_no_fe, pseudocount=pseudoc)

# ---------- Save ----------


In [4]:
df_out.head()

,material_no,C (%),Si (%),Mn (%),S (%),P (%),Cr (%),Ni (%),Cu (%),Mo (%),Fe (%),C (%)_clr,Si (%)_clr,Mn (%)_clr,S (%)_clr,P (%)_clr,Cr (%)_clr,Ni (%)_clr,Cu (%)_clr,Mo (%)_clr
0,1,0.030,0.150,0.005,0.023,0.145,0.0,0.0,0.02,0.0,99.627,5.219872,6.829310,3.428114,4.954169,6.795409,-10.680427,-10.680427,4.814407,-10.680427
1,3,0.020,0.150,0.033,0.022,0.195,0.0,0.0,0.03,0.0,99.550,4.514197,6.529100,5.014973,4.609508,6.791464,-10.792968,-10.792968,4.919662,-10.792968
2,4,0.020,0.150,0.033,0.022,0.195,0.0,0.0,0.03,0.0,99.550,4.514197,6.529100,5.014973,4.609508,6.791464,-10.792968,-10.792968,4.919662,-10.792968
3,7,0.016,0.100,0.029,0.018,0.160,0.0,0.0,0.00,0.0,99.677,6.312757,8.145338,6.907464,6.430540,8.615342,-9.102860,-9.102860,-9.102860,-9.102860
4,8,0.017,0.125,0.041,0.018,0.106,0.0,0.0,0.00,0.0,99.693,6.371705,8.366805,7.252063,6.428863,8.201930,-9.155342,-9.155342,-9.155342,-9.155342


In [5]:
df_out.to_csv('composition_clr.csv', index=False)